In [1]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


In [2]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/eda/train_data_v1.csv")


In [3]:
clean_df.head()

,caption,product,image_file_path,image_parent_folder,caption_word_count,img_ext,full_path
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,20,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,14,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,16,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,19,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,30,qXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...


In [4]:
image_size = 384
preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 


In [5]:
def test_feature(caption_col,clip_model):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/eda/train_data_v1.csv",
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.image_col,caption_col,CONFIG.test_batch_size,preprocess,CONFIG.global_random_state,test=False,use_all=True,tokenize=False)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred    
    return image_embed,text_embed

In [6]:
col_to_test =  "caption"
checkpoint = "/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/save_checkpoint_5_tues.pth"
clip_model =ALIPRODUCT_BLIP(checkpoint,image_size)
image_embed,text_embed = test_feature(col_to_test,clip_model)



load checkpoint from /home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/save_checkpoint_5_tues.pth


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0:   4%|▍         | 1109/25624 [1:06:21<24:26:53,  3.59s/it]

TypeError: zip() argument after * must be an iterable, not NoneType

In [ ]:
single_pair_cosine = np.array(list((txt @ img.T).item() for txt ,img in tqdm(zip(text_embed,image_embed))))

50000it [00:00, 144393.37it/s]


In [ ]:
clean_df["single_pair_cosine"] = single_pair_cosine

In [ ]:
clean_df.to_csv("../data/train_data_v2.csv")